In [1]:
import os
import numpy as np
import efficientnet.tfkeras
from tensorflow.keras.models import load_model
from tensorflow.keras import models
import tensorflow as tf
from focal_loss import BinaryFocalLoss

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]=""

In [3]:
model_dir ='/media/tohn/SSD/ModelTrainByImages/R2_8/models/B5R2b5_5FP_1FC_fold8_1.h5' #เปลี่ยน model ต้องเอา R2 ## 
model = load_model(model_dir)
height = width = model.input_shape[1]

x = model.get_layer('head_pooling').output
prediction_layer = model.output
model2 = models.Model(inputs= model.input, outputs=[prediction_layer,x]) 

In [4]:
#ไม่ต้องเปลี่ยนก็ได้ เพราะเอา Label มาใช้เฉยๆ 
# validation
import pandas as pd
base_dir  = '/media/tohn/SSD/Images/Image1/validation/'
dataframe = pd.read_csv( '/home/yupaporn/codes/USAI/Validationdf_fold1.csv')
validation_dir = os.path.join(base_dir, 'validation')

#Train
train_df = pd.read_csv( '/home/yupaporn/codes/USAI/Traindf_fold1.csv')
base_dir0 = '/media/tohn/SSD/Images/Image1/'
os.chdir(base_dir0)
train_dir = os.path.join(base_dir0, 'train')

In [5]:
import pandas as pd
df0 = pd.read_csv (r'/home/yupaporn/codes/USAI/Traindf_fold8.csv') #Dataset
print(df0 .shape)
dataframe = df0[(df0['Path Crop']!='None' )&(df0['Path Crop']!='Nan')]
# a = df01[df01['Case'].between(1, 10)]
# dataframe = df01[df01['Abs Position']!='P8']
print(dataframe.shape)
print('Normal: ',dataframe[dataframe['Class']=='Normal'].shape)
print('Abnormal: ',dataframe[dataframe['Class']=='Abnormal'].shape)
dataframe.head(5)

(4601, 25)
(4601, 25)
Normal:  (3006, 25)
Abnormal:  (1595, 25)


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Case,Abs Position,Sub Position,Class,Sub_class,Path Full,...,left,top,width,height,Rleft,Rtop,Rwidth,Rheight,FileName,filename
0,0,0,0,0,15,P3,P31,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,35.0,96.0,694.0,584.0,0.036832,0.107041,0.712707,0.649402,AB01 P3-1 C015.JPG,AB01 P3-1 C015.JPG
1,1,1,1,1,15,P1,P1,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,141.0,96.0,586.0,614.0,0.143898,0.107041,0.595628,0.683267,AB01 P1 C015.JPG,AB01 P1 C015.JPG
2,2,2,2,2,15,P2,P2,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,242.0,101.0,491.0,451.0,0.245009,0.113017,0.497278,0.501992,AB01 P2 C015.JPG,AB01 P2 C015.JPG
3,3,3,3,3,15,P4,P42,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,154.0,110.0,714.0,611.0,0.155235,0.122977,0.718412,0.679283,AB01 P4-2 C015.JPG,AB01 P4-2 C015.JPG
4,4,4,4,4,15,P4,P41,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,152.0,146.0,609.0,519.0,0.152057,0.162817,0.608229,0.577689,AB01 P4-1 C015.JPG,AB01 P4-1 C015.JPG


In [6]:
# dataframe.info()

In [6]:
batch_size = 64
epochs = 10

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      brightness_range=[0.5,1.5],
      shear_range=0.4,
      zoom_range=0.2,
      horizontal_flip=False,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_dataframe(
        dataframe = dataframe,
        directory = train_dir,
        x_col = 'Path Crop',
        y_col = 'Views',
        target_size = (height, width),
        batch_size=batch_size,
        color_mode= 'rgb',
        class_mode='categorical')

#label
labels = (train_generator.class_indices)
labels = dict((v,k.replace("C","")) for k,v in labels.items())
print(labels)

Found 4601 validated image filenames belonging to 5 classes.
{0: 'FP-A', 1: 'FP-B', 2: 'FP-', 3: 'FP-D', 4: 'FP-E'}


In [7]:
from tensorflow.keras.preprocessing import image
def predict_image(img_path):
    # Read the image and resize it
    img = image.load_img(img_path, target_size=(height, width))
    # Convert it to a Numpy array with target shape.
    x = image.img_to_array(img)
    # Reshape
    x = x.reshape((1,) + x.shape)
    x /= 255.
    result = model2.predict([x])
    result1 = result[0][0]
    result2 = result[1][0]

    return result1, result2

#Predict
pred_list = list()
prob_list = list()
vectorlist = list()
img_path=dataframe['Path Crop'].tolist()
for i in range(0,len(img_path)):
    result1,result2 = predict_image(img_path[i])
    result = np.argmax(result1)
    pred_list.append(labels[result])
    prob_list.append(result1[result])
    vectorlist.append(result2.tolist())
    
dataframe['category'] = pred_list
dataframe['Prob'] = prob_list

In [8]:
vector_array = np.array(vectorlist)
vector_array

array([[ 0.44917366, -0.20442782, -0.19914395, ...,  0.09513656,
         0.05301182,  0.38721204],
       [ 0.35900787,  0.72091186,  0.31571025, ...,  0.00607038,
        -0.01608799,  0.2720716 ],
       [ 0.05514898,  0.5115236 , -0.00377148, ..., -0.07041273,
        -0.22297014,  0.4384872 ],
       ...,
       [-0.18228662, -0.2085792 ,  0.43211216, ...,  0.67089337,
         0.49563551, -0.11058271],
       [-0.17771965, -0.21117987, -0.02204037, ...,  0.25023362,
         0.63147593, -0.14395444],
       [-0.02251811, -0.21400946, -0.22061652, ...,  0.58949429,
        -0.22045222,  0.72196424]])

In [9]:
dataset = pd.DataFrame(vector_array[:, :])
dataset

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,0.449174,-0.204428,-0.199144,-0.157085,-0.118373,-0.122697,-0.195651,0.409199,0.039756,0.521029,...,-0.005710,-0.193740,-0.068002,0.622498,-0.208558,0.898302,0.243205,0.095137,0.053012,0.387212
1,0.359008,0.720912,0.315710,0.268273,0.106144,0.100688,0.846621,0.480937,0.157924,0.247229,...,0.161412,-0.168381,-0.180546,-0.088734,-0.190041,0.451685,0.358777,0.006070,-0.016088,0.272072
2,0.055149,0.511524,-0.003771,0.241515,0.240965,-0.129346,0.678433,-0.003770,-0.143136,-0.078084,...,0.107142,-0.182571,-0.227701,-0.189710,-0.197617,0.107274,0.133976,-0.070413,-0.222970,0.438487
3,0.061421,-0.197331,-0.155033,-0.128618,-0.084117,-0.110935,-0.195612,0.350846,-0.011851,0.139456,...,0.175199,-0.106868,-0.063115,0.721309,-0.211585,0.424843,0.262484,0.229328,-0.024867,0.186694
4,0.544007,-0.196851,-0.154777,-0.131862,-0.012534,0.045621,-0.210399,0.362736,-0.048656,0.457716,...,0.263864,-0.113276,0.000620,0.759631,-0.221963,0.417165,0.224456,0.247668,0.056598,0.158641
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4596,0.601950,-0.215200,-0.210370,-0.084321,-0.041753,-0.034182,-0.205843,0.408583,-0.135126,0.623134,...,0.843900,0.065779,-0.040122,0.518132,-0.160655,0.743273,0.685318,0.032199,-0.094300,0.028544
4597,0.705719,-0.216591,-0.155282,-0.144300,0.076284,0.139908,-0.180171,0.417639,0.011849,0.538739,...,0.532851,-0.024162,0.055639,0.727655,-0.148911,0.498000,0.623234,-0.094777,-0.000488,0.484960
4598,-0.182287,-0.208579,0.432112,0.227209,0.694599,-0.129581,0.495339,0.077959,0.950621,-0.204977,...,0.138278,0.730541,1.074448,0.186817,0.927064,-0.109220,-0.124952,0.670893,0.495636,-0.110583
4599,-0.177720,-0.211180,-0.022040,0.163384,0.268721,-0.158802,-0.198402,0.095069,0.184693,-0.048553,...,-0.017296,0.052770,0.442896,0.122819,0.521852,-0.091544,-0.129625,0.250234,0.631476,-0.143954


In [10]:
a = dataframe['Class'].tolist()
b = dataframe['Sub Position'].tolist()
c = dataframe['Views'].tolist()
d = dataframe['Sub_class'].tolist()
dataset['Class'] = a 
dataset['SubPosition'] = b
dataset['Views'] = c
dataset['Sub_class'] = d

In [11]:
dataset

,0,1,2,3,4,5,6,7,8,9,...,2042,2043,2044,2045,2046,2047,Class,SubPosition,Views,Sub_class
0,0.449174,-0.204428,-0.199144,-0.157085,-0.118373,-0.122697,-0.195651,0.409199,0.039756,0.521029,...,-0.208558,0.898302,0.243205,0.095137,0.053012,0.387212,Abnormal,P31,FP-B,AB01
1,0.359008,0.720912,0.315710,0.268273,0.106144,0.100688,0.846621,0.480937,0.157924,0.247229,...,-0.190041,0.451685,0.358777,0.006070,-0.016088,0.272072,Abnormal,P1,FP-A,AB01
2,0.055149,0.511524,-0.003771,0.241515,0.240965,-0.129346,0.678433,-0.003770,-0.143136,-0.078084,...,-0.197617,0.107274,0.133976,-0.070413,-0.222970,0.438487,Abnormal,P2,FP-A,AB01
3,0.061421,-0.197331,-0.155033,-0.128618,-0.084117,-0.110935,-0.195612,0.350846,-0.011851,0.139456,...,-0.211585,0.424843,0.262484,0.229328,-0.024867,0.186694,Abnormal,P42,FP-B,AB01
4,0.544007,-0.196851,-0.154777,-0.131862,-0.012534,0.045621,-0.210399,0.362736,-0.048656,0.457716,...,-0.221963,0.417165,0.224456,0.247668,0.056598,0.158641,Abnormal,P41,FP-B,AB01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4596,0.601950,-0.215200,-0.210370,-0.084321,-0.041753,-0.034182,-0.205843,0.408583,-0.135126,0.623134,...,-0.160655,0.743273,0.685318,0.032199,-0.094300,0.028544,Normal,P32,FP-B,Normal
4597,0.705719,-0.216591,-0.155282,-0.144300,0.076284,0.139908,-0.180171,0.417639,0.011849,0.538739,...,-0.148911,0.498000,0.623234,-0.094777,-0.000488,0.484960,Normal,P42,FP-B,Normal
4598,-0.182287,-0.208579,0.432112,0.227209,0.694599,-0.129581,0.495339,0.077959,0.950621,-0.204977,...,0.927064,-0.109220,-0.124952,0.670893,0.495636,-0.110583,Normal,P52,FP-C,Normal
4599,-0.177720,-0.211180,-0.022040,0.163384,0.268721,-0.158802,-0.198402,0.095069,0.184693,-0.048553,...,0.521852,-0.091544,-0.129625,0.250234,0.631476,-0.143954,Normal,P61,FP-C,Normal


In [12]:
os.chdir( '/home/yupaporn/codes/USAI/')
testdf = dataset
testdf.to_csv('FVtrain_5FP_fold8_1.csv') #เปลี่ยนชื่อไฟล์